# ImageBackend

In [1]:
from wholeslidedata.image.backend import WholeSlideImageBackend
from wholeslidedata.interoperability.asap.backend import AsapWholeSlideImageBackend
from wholeslidedata import WholeSlideImage
from wholeslidedata.iterators import create_batch_iterator

For opening Whole-slide images, the wholeslidedata package relies on well-known and established python libraries such as multiresolutionimageinterface (ASAP), openslide (openslide-python) and pyvips.
At least one of these backends needs to be installed. 

:::{seealso}
Please see the installation instructions for the image backends [here]().
:::



:::{warning}
Monochrome mask files can **not** be opened with the openslide image backend
:::



The image backend can be specified via the 'backend' argument when instantiating WholeSlideImage

In [2]:
wsi = WholeSlideImage('/tmp/TCGA-21-5784-01Z-00-DX1.tif', backend=AsapWholeSlideImageBackend)
wsi._backend

<wholeslidedata.interoperability.asap.backend.AsapWholeSlideImageBackend; proxy of <Swig Object of type 'MultiResolutionImage *' at 0x7f90335c9d50> >

In [10]:
wsi = WholeSlideImage('/tmp/TCGA-21-5784-01Z-00-DX1.tif', backend="openslide")
wsi._backend

OpenSlideWholeSlideImageBackend('/tmp/TCGA-21-5784-01Z-00-DX1.tif')

## Specifying Backend in user config

:::{note}
There are multiple ways to specify the image backend in the user config. The most straightforward is globally. In rare situations, you can also specify the image backend for each image separately
:::

### Global

In [29]:
user_config_global_backend = './configs/user_config_global_backend.yml'
!cat {user_config_global_backend}

wholeslidedata:
    default:
        image_backend: ${$wholeslidedata.interoperability.asap.backend.AsapWholeSlideImageBackend}
        
        yaml_source:
            training:
                - wsi: 
                    path: /tmp/TCGA-21-5784-01Z-00-DX1.tif
                  wsa: 
                    path: /tmp/TCGA-21-5784-01Z-00-DX1.xml       
            validation:
                - wsi: 
                    path: /tmp/TCGA-21-5784-01Z-00-DX1.tif
                  wsa: 
                    path: /tmp/TCGA-21-5784-01Z-00-DX1.xml 
        labels:
            stroma: 1
            tumor: 2
            lymphocytes: 3
            
        batch_shape:
            batch_size: 4
            spacing: 1.0
            shape: [512, 512, 3]

In [19]:
training_iterator = create_batch_iterator(mode='training', user_config=user_config_global_backend)
validation_iterator = create_batch_iterator(mode='validation', user_config=user_config_global_backend)
print(training_iterator.dataset['TCGA-21-5784-01Z-00-DX1']['images'][0]._backend)
print(validation_iterator.dataset['TCGA-21-5784-01Z-00-DX1']['images'][0]._backend)
training_iterator.stop()
validation_iterator.stop()

<wholeslidedata.interoperability.asap.backend.AsapWholeSlideImageBackend; proxy of <Swig Object of type 'MultiResolutionImage *' at 0x7f37c14cf150> >
<wholeslidedata.interoperability.asap.backend.AsapWholeSlideImageBackend; proxy of <Swig Object of type 'MultiResolutionImage *' at 0x7f37aa16bc00> >




### Per image


In [30]:
user_config_backend_per_image = './configs/user_config_backend_per_image.yml'
!cat {user_config_backend_per_image}

wholeslidedata:
    default:
        yaml_source:
            training:
                - wsi: 
                    path: /tmp/TCGA-21-5784-01Z-00-DX1.tif
                    image_backend: ${$wholeslidedata.interoperability.openslide.backend.OpenSlideWholeSlideImageBackend}
                  wsa: 
                    path: /tmp/TCGA-21-5784-01Z-00-DX1.xml       
            validation:
                - wsi: 
                    path: /tmp/TCGA-21-5784-01Z-00-DX1.tif
                    image_backend: ${$wholeslidedata.interoperability.asap.backend.AsapWholeSlideImageBackend}
                  wsa: 
                    path: /tmp/TCGA-21-5784-01Z-00-DX1.xml 
        labels:
            stroma: 1
            tumor: 2
            lymphocytes: 3
            
        batch_shape:
            batch_size: 4
            spacing: 1.0
            shape: [512, 512, 3]

In [27]:
training_iterator = create_batch_iterator(mode='training', user_config=user_config_backend_per_image)
validation_iterator = create_batch_iterator(mode='validation', user_config=user_config_backend_per_image)
print(training_iterator.dataset['TCGA-21-5784-01Z-00-DX1']['images'][0]._backend)
print(validation_iterator.dataset['TCGA-21-5784-01Z-00-DX1']['images'][0]._backend)
training_iterator.stop()
validation_iterator.stop()

OpenSlideWholeSlideImageBackend('/tmp/TCGA-21-5784-01Z-00-DX1.tif')
<wholeslidedata.interoperability.asap.backend.AsapWholeSlideImageBackend; proxy of <Swig Object of type 'MultiResolutionImage *' at 0x7f37a7738900> >
